In [8]:
# Run in your environment (Python >=3.8). Requires: pandas, numpy, scikit-learn, xgboost, joblib
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import joblib
import os

# ------------- config -------------
DATA_PATH = "PRSA_Aotizhongxin_cleaned.csv"   # replace with your file path
TARGET = "PM2.5"
LAGS = 3
HORIZONS = [1, 3, 6, 12, 24]
TRAIN_FRAC = 0.8
RESULT_DIR = "results_multi_horizon"
os.makedirs(RESULT_DIR, exist_ok=True)
# -----------------------------------

df = pd.read_csv(DATA_PATH, parse_dates=True, index_col=0)  # index should be datetime

def make_supervised(df, target_col, lags=3, horizon=1):
    df_sup = df.copy()
    # create lag features for all columns
    for lag in range(1, lags + 1):
        df_sup = df_sup.assign(**{f"{c}_lag{lag}": df[c].shift(lag) for c in df.columns})

    # target shifted up by horizon (predict future)
    df_sup[f"y_h{horizon}"] = df_sup[target_col].shift(-horizon)
    df_sup = df_sup.dropna()

    X = df_sup.drop(columns=[f"y_h{horizon}"])
    y = df_sup[f"y_h{horizon}"]
    return X, y

def train_test_split_time(X, y, train_frac=0.8):
    n = len(X)
    split = int(n * train_frac)
    X_train, X_test = X.iloc[:split], X.iloc[split:]
    y_train, y_test = y.iloc[:split], y.iloc[split:]
    return X_train, X_test, y_train, y_test

def ensure_numeric(X_train, X_test):
    """
    Convert object dtypes to numeric/boolean. Coerce invalid -> NaN, then fillna(0).
    Returns numeric X_train, X_test (floats).
    """
    X_train = X_train.copy()
    X_test = X_test.copy()

    # find object columns
    obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    if obj_cols:
        print("Converting object columns to numeric:", obj_cols)
    for c in obj_cols:
        # try numeric conversion
        X_train[c] = pd.to_numeric(X_train[c], errors='coerce')
        X_test[c] = pd.to_numeric(X_test[c], errors='coerce')

        # if still object (unlikely), try bool mapping
        if X_train[c].dtype == 'object' or X_test[c].dtype == 'object':
            X_train[c] = X_train[c].map({'True':1, 'False':0, True:1, False:0}).astype('float')
            X_test[c] = X_test[c].map({'True':1, 'False':0, True:1, False:0}).astype('float')

    # After conversion, fill NaNs with 0 (safe for one-hot and numeric lag features)
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    # finally cast to float (XGBoost and sklearn expect numeric arrays)
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)
    return X_train, X_test

models = {
    "Naive": None,  # handled separately
    "Linear": make_pipeline(StandardScaler(), LinearRegression()),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(n_estimators=200, random_state=42, n_jobs=4, verbosity=0)
}

results = []
per_hour_errors = {}  # store arrays for paired tests

for h in HORIZONS:
    print(f"\n=== Horizon: {h} hours ===")
    X, y = make_supervised(df, TARGET, lags=LAGS, horizon=h)
    X_train, X_test, y_train, y_test = train_test_split_time(X, y, TRAIN_FRAC)

    # Ensure numeric dtypes before any model uses X (including naive baseline)
    X_train_num, X_test_num = ensure_numeric(X_train, X_test)

    # Naive baseline: PM2.5_lag1 (ensure it's numeric)
    naive_pred = X_test_num[f"{TARGET}_lag1"].values
    naive_rmse = mean_squared_error(y_test, naive_pred) ** 0.5
    print("Naive RMSE:", naive_rmse)

    results.append({"horizon": h, "model": "Naive", "rmse": float(naive_rmse)})
    per_hour_errors[(h, "Naive")] = (y_test.values - naive_pred)

    for mname, m in models.items():
        if mname == "Naive":
            continue
        print(" Training:", mname)

        # For models that include preprocessing (e.g., Linear pipeline), use original Xframes but numeric
        # For RandomForest and XGBoost we pass numeric arrays
        if mname == "Linear":
            # Linear pipeline expects DataFrame-like input; using the numeric DataFrames is fine
            m.fit(X_train_num, y_train)
            yhat = m.predict(X_test_num)
        else:
            # RandomForest and XGBoost also accept DataFrames or arrays
            m.fit(X_train_num, y_train)
            yhat = m.predict(X_test_num)

        rmse = mean_squared_error(y_test, yhat) ** 0.5
        print(f"  {mname} RMSE: {rmse:.4f}")

        results.append({"horizon": h, "model": mname, "rmse": float(rmse)})
        per_hour_errors[(h, mname)] = (y_test.values - yhat)

        # optional: save model
        joblib.dump(m, os.path.join(RESULT_DIR, f"{mname}_h{h}.joblib"))

# save summary
pd.DataFrame(results).pivot(index="model", columns="horizon", values="rmse") \
    .to_csv(os.path.join(RESULT_DIR, "rmse_table.csv"))

# save per-hour errors (for statistical tests later)
joblib.dump(per_hour_errors, os.path.join(RESULT_DIR, "per_hour_errors.joblib"))

print("Done. Results saved to:", RESULT_DIR)



=== Horizon: 1 hours ===
Converting object columns to numeric: ['wd_E_lag1', 'wd_ENE_lag1', 'wd_ESE_lag1', 'wd_N_lag1', 'wd_NE_lag1', 'wd_NNE_lag1', 'wd_NNW_lag1', 'wd_NW_lag1', 'wd_S_lag1', 'wd_SE_lag1', 'wd_SSE_lag1', 'wd_SSW_lag1', 'wd_SW_lag1', 'wd_W_lag1', 'wd_WNW_lag1', 'wd_WSW_lag1', 'wd_E_lag2', 'wd_ENE_lag2', 'wd_ESE_lag2', 'wd_N_lag2', 'wd_NE_lag2', 'wd_NNE_lag2', 'wd_NNW_lag2', 'wd_NW_lag2', 'wd_S_lag2', 'wd_SE_lag2', 'wd_SSE_lag2', 'wd_SSW_lag2', 'wd_SW_lag2', 'wd_W_lag2', 'wd_WNW_lag2', 'wd_WSW_lag2', 'wd_E_lag3', 'wd_ENE_lag3', 'wd_ESE_lag3', 'wd_N_lag3', 'wd_NE_lag3', 'wd_NNE_lag3', 'wd_NNW_lag3', 'wd_NW_lag3', 'wd_S_lag3', 'wd_SE_lag3', 'wd_SSE_lag3', 'wd_SSW_lag3', 'wd_SW_lag3', 'wd_W_lag3', 'wd_WNW_lag3', 'wd_WSW_lag3']
Naive RMSE: 30.617581499440146
 Training: Linear
  Linear RMSE: 17.4797
 Training: RandomForest
  RandomForest RMSE: 18.1745
 Training: XGBoost
  XGBoost RMSE: 19.3884

=== Horizon: 3 hours ===
Converting object columns to numeric: ['wd_E_lag1', 'wd_E

In [9]:
# save as analysis_and_plots.py and run it
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_rel, wilcoxon

plt.rcParams.update({"figure.dpi": 150})

RESULT_DIR = "results_multi_horizon"
PLOTS_DIR = os.path.join(RESULT_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# 1) load rmse table and per-hour errors
rmse_table = pd.read_csv(os.path.join(RESULT_DIR, "rmse_table.csv"), index_col=0)
per_hour_errors = joblib.load(os.path.join(RESULT_DIR, "per_hour_errors.joblib"))

# show RMSE table
print("\nRMSE table (models rows, horizons columns):")
print(rmse_table)

# 2) RMSE vs horizon plot (line)
plt.figure()
for model in rmse_table.index:
    plt.plot(rmse_table.columns.astype(int), rmse_table.loc[model].values, marker='o', label=model)
plt.xlabel("Horizon (hours)")
plt.ylabel("RMSE (µg/m³)")
plt.title("RMSE vs Forecast Horizon")
plt.legend()
plt.grid(alpha=0.25)
plt.savefig(os.path.join(PLOTS_DIR, "rmse_vs_horizon.png"))
plt.close()

# 3) For each horizon: Actual vs Predicted scatter for top models (Naive, Linear, RandomForest)
# We use per_hour_errors to reconstruct preds as actual - error. Need y_test for actuals: reconstruct from errors keys (they store only errors arrays)
# The code assumes all per_hour_errors[(h,model)] have same length and same ordering for a given horizon.
# We'll choose first horizon to demonstrate and do Residual histogram/time series for all.
for h in sorted({k[0] for k in per_hour_errors.keys()}):
    h = int(h)
    print(f"\nProcessing horizon {h}h")
    # gather models available for this horizon
    models = [k[1] for k in per_hour_errors.keys() if k[0] == h]
    # reconstruct actuals using Naive: y = naive_error + naive_pred but we don't have naive_pred directly
    # Instead we can compute relative errors only (difference between models), and plot error distributions.
    # We'll produce: residual histogram per model and paired test (Naive vs RF)
    plt.figure(figsize=(7,4))
    for m in ["Naive", "Linear", "RandomForest", "XGBoost"]:
        key = (h, m)
        if key not in per_hour_errors: 
            continue
        errs = np.array(per_hour_errors[key]).ravel()
        plt.hist(errs, bins=80, alpha=0.5, density=False, label=f"{m} (n={len(errs)})")
    plt.xlabel("Residual (actual - predicted)")
    plt.title(f"Residual histogram (Horizon {h}h)")
    plt.legend()
    plt.savefig(os.path.join(PLOTS_DIR, f"residual_hist_h{h}.png"))
    plt.close()

    # residual time-series sample (first 1000 points)
    plt.figure(figsize=(9,3))
    for m in ["Naive", "RandomForest", "Linear"]:
        key = (h, m)
        if key not in per_hour_errors: 
            continue
        errs = np.array(per_hour_errors[key]).ravel()
        plt.plot(errs[:1000], label=m, alpha=0.8)
    plt.xlabel("Test index (sample)")
    plt.ylabel("Residual")
    plt.title(f"Residual time series (first 1000) - Horizon {h}h")
    plt.legend()
    plt.savefig(os.path.join(PLOTS_DIR, f"residual_ts_h{h}.png"))
    plt.close()

    # Paired statistical tests: (Naive_error) - (RF_error) per hour => positive means RF smaller error
    if (h, "Naive") in per_hour_errors and (h, "RandomForest") in per_hour_errors:
        naive_err = np.array(per_hour_errors[(h, "Naive")]).ravel()
        rf_err = np.array(per_hour_errors[(h, "RandomForest")]).ravel()
        # Ensure same length
        n = min(len(naive_err), len(rf_err))
        naive_err, rf_err = naive_err[:n], rf_err[:n]
        diff = naive_err - rf_err  # positive -> RF better
        tstat, pval = ttest_rel(naive_err, rf_err)
        # Wilcoxon (two-sided) - requires non-zero length and matched pairs
        try:
            wstat, wp = wilcoxon(naive_err - rf_err)
        except Exception as e:
            wstat, wp = np.nan, np.nan

        print(f"H{h}h paired t-test Naive vs RF: t={tstat:.3f}, p={pval:.3e}; Wilcoxon: stat={wstat}, p={wp:.3e}")

# 4) Feature importance (RandomForest) - load saved RF models if present
import joblib, glob
rf_files = glob.glob(os.path.join(RESULT_DIR, "RandomForest_h*.joblib"))
if rf_files:
    for f in rf_files:
        # extract horizon from filename
        basename = os.path.basename(f)
        h = int(basename.split("_h")[-1].split(".joblib")[0])
        model = joblib.load(f)
        if hasattr(model, "feature_importances_"):
            # we need the feature names saved in rmse run: we will rebuild X for that horizon to get column names
            # load df again (assume same DATA_PATH used earlier)
            df = pd.read_csv("PRSA_Aotizhongxin_cleaned.csv", parse_dates=True, index_col=0)
            # create supervised to get X columns
            def make_supervised_local(df, target_col, lags=3, horizon=1):
                df_sup = df.copy()
                for lag in range(1, lags+1):
                    df_sup = df_sup.assign(**{f"{c}_lag{lag}": df[c].shift(lag) for c in df.columns})
                df_sup[f"y_h{horizon}"] = df_sup[target_col].shift(-horizon)
                df_sup = df_sup.dropna()
                X = df_sup.drop(columns=[f"y_h{horizon}"])
                return X
            X = make_supervised_local(df, "PM2.5", lags=3, horizon=h)
            # convert object cols same as training
            obj_cols = X.select_dtypes(include=['object']).columns
            for c in obj_cols:
                X[c] = pd.to_numeric(X[c], errors='coerce').fillna(0)
            # now get importance
            fi = model.feature_importances_
            idx = np.argsort(fi)[-30:]  # top 30
            top_feats = X.columns[idx]
            plt.figure(figsize=(6,8))
            plt.barh(range(len(idx)), fi[idx])
            plt.yticks(range(len(idx)), top_feats)
            plt.xlabel("Importance")
            plt.title(f"RandomForest top features (H{h}h)")
            plt.tight_layout()
            plt.savefig(os.path.join(PLOTS_DIR, f"rf_feature_importance_h{h}.png"))
            plt.close()

print("\nAll plots saved to:", PLOTS_DIR)



RMSE table (models rows, horizons columns):
                      1          3          6         12         24
model                                                              
Linear        17.479715  36.345979  50.556133  62.807161  72.266227
Naive         30.617581  45.465007  60.073345  75.447012  90.928599
RandomForest  18.174493  36.517867  51.246618  63.689615  73.403836
XGBoost       19.388413  39.500271  53.962063  70.344928  79.663455

Processing horizon 1h
H1h paired t-test Naive vs RF: t=1.007, p=3.138e-01; Wilcoxon: stat=10251246.0, p=1.962e-33

Processing horizon 3h
H3h paired t-test Naive vs RF: t=3.623, p=2.937e-04; Wilcoxon: stat=8833302.5, p=1.267e-92

Processing horizon 6h
H6h paired t-test Naive vs RF: t=3.027, p=2.480e-03; Wilcoxon: stat=8219865.5, p=1.861e-127

Processing horizon 12h
H12h paired t-test Naive vs RF: t=2.994, p=2.761e-03; Wilcoxon: stat=8198355.0, p=1.268e-128

Processing horizon 24h
H24h paired t-test Naive vs RF: t=10.089, p=8.972e-24; Wilcoxo

In [11]:
# tune_models_fixed.py
# Hyperparameter tuning for RandomForest and XGBoost across multiple horizons.
# This variant forces single-process CV to avoid joblib serialization errors.
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.stats import randint, uniform

# ---------- CONFIG ----------
DATA_PATH = "PRSA_Aotizhongxin_cleaned.csv"
TARGET = "PM2.5"
LAGS = 3
HORIZONS = [1, 3, 6, 12, 24]
TRAIN_FRAC = 0.8
RESULT_DIR = "results_multi_horizon"
TUNE_DIR = os.path.join(RESULT_DIR, "tuned_models")
os.makedirs(TUNE_DIR, exist_ok=True)
RANDOM_STATE = 42
RF_ITER = 20     # reduce iterations to keep total time reasonable
XGB_ITER = 25
N_JOBS_SEARCH = 1   # IMPORTANT: run CV single-threaded to avoid pickling issues
N_JOBS_EST = 1      # estimator-level parallelism inside CV (keep 1 to avoid nested parallelism)
N_SPLITS = 3
# ----------------------------

def make_supervised(df, target_col, lags=3, horizon=1):
    df_sup = df.copy()
    for lag in range(1, lags + 1):
        df_sup = df_sup.assign(**{f"{c}_lag{lag}": df[c].shift(lag) for c in df.columns})
    df_sup[f"y_h{horizon}"] = df_sup[target_col].shift(-horizon)
    df_sup = df_sup.dropna()
    X = df_sup.drop(columns=[f"y_h{horizon}"])
    y = df_sup[f"y_h{horizon}"]
    return X, y

def train_test_split_time(X, y, train_frac=0.8):
    n = len(X)
    split = int(n * train_frac)
    return X.iloc[:split], X.iloc[split:], y.iloc[:split], y.iloc[split:]

def ensure_numeric(X_train, X_test):
    X_train = X_train.copy()
    X_test = X_test.copy()
    obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    for c in obj_cols:
        X_train[c] = pd.to_numeric(X_train[c], errors='coerce')
        X_test[c] = pd.to_numeric(X_test[c], errors='coerce')
        if X_train[c].dtype == 'object' or X_test[c].dtype == 'object':
            X_train[c] = X_train[c].map({'True':1, 'False':0, True:1, False:0}).astype('float')
            X_test[c]  = X_test[c].map({'True':1, 'False':0, True:1, False:0}).astype('float')
    X_train = X_train.fillna(0).astype(float)
    X_test  = X_test.fillna(0).astype(float)
    return X_train, X_test

# load df
df = pd.read_csv(DATA_PATH, parse_dates=True, index_col=0)

# Try to load previous rmse table (optional)
prev_rmse_path = os.path.join(RESULT_DIR, "rmse_table.csv")
prev_rmse = pd.read_csv(prev_rmse_path, index_col=0) if os.path.exists(prev_rmse_path) else None

tuning_summary = []

for h in HORIZONS:
    print("\n" + "="*60)
    print(f"Tuning for horizon {h} hour(s)")
    X, y = make_supervised(df, TARGET, lags=LAGS, horizon=h)
    X_train, X_test, y_train, y_test = train_test_split_time(X, y, TRAIN_FRAC)
    X_train_num, X_test_num = ensure_numeric(X_train, X_test)

    # get previous RMSEs if available, else compute defaults quickly
    if prev_rmse is not None:
        try:
            prev_lin = float(prev_rmse.loc["Linear", str(h)])
            prev_rf  = float(prev_rmse.loc["RandomForest", str(h)])
            prev_xgb = float(prev_rmse.loc["XGBoost", str(h)])
            prev_naive = float(prev_rmse.loc["Naive", str(h)])
        except Exception:
            prev_lin = prev_rf = prev_xgb = prev_naive = np.nan
    else:
        # compute naive + defaults
        naive_pred = X_test_num[f"{TARGET}_lag1"].values
        prev_naive = mean_squared_error(y_test, naive_pred)**0.5
        from sklearn.pipeline import make_pipeline
        from sklearn.preprocessing import StandardScaler
        from sklearn.linear_model import LinearRegression
        lin = make_pipeline(StandardScaler(), LinearRegression())
        lin.fit(X_train_num, y_train)
        prev_lin = mean_squared_error(y_test, lin.predict(X_test_num))**0.5
        rf_def = RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=1)
        rf_def.fit(X_train_num, y_train)
        prev_rf = mean_squared_error(y_test, rf_def.predict(X_test_num))**0.5
        xgb_def = xgb.XGBRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=1, verbosity=0, objective='reg:squarederror')
        xgb_def.fit(X_train_num, y_train)
        prev_xgb = mean_squared_error(y_test, xgb_def.predict(X_test_num))**0.5

    print(f"Previous RMSEs (approx): Linear={prev_lin:.4f}, RF={prev_rf:.4f}, XGB={prev_xgb:.4f}, Naive={prev_naive:.4f}")

    # ----------------- Random Forest Tuning -----------------
    rf = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS_EST)
    rf_param_dist = {
        "n_estimators": randint(100, 600),
        "max_depth": randint(4, 30),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 20),
        "max_features": ["auto", "sqrt", "log2", 0.3, 0.5, 0.8]
    }
    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    rf_search = RandomizedSearchCV(
        rf,
        rf_param_dist,
        n_iter=RF_ITER,
        scoring="neg_mean_squared_error",
        cv=tscv,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS_SEARCH,
        verbose=1
    )
    print("Running RandomForest RandomizedSearchCV (single-process)...")
    rf_search.fit(X_train_num, y_train)
    best_rf = rf_search.best_estimator_
    rf_pred = best_rf.predict(X_test_num)
    rf_rmse_tuned = mean_squared_error(y_test, rf_pred)**0.5
    print(f"RF tuned RMSE (H{h}): {rf_rmse_tuned:.4f}")
    joblib.dump(best_rf, os.path.join(TUNE_DIR, f"RandomForest_h{h}_tuned.joblib"))

    # ----------------- XGBoost Tuning -----------------
    xg = xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS_EST, verbosity=0, objective='reg:squarederror')
    xgb_param_dist = {
        "n_estimators": randint(100, 800),
        "max_depth": randint(2, 12),
        "learning_rate": uniform(0.01, 0.4),
        "subsample": uniform(0.5, 0.5),
        "colsample_bytree": uniform(0.4, 0.6),
        "gamma": uniform(0, 5),
        "reg_alpha": uniform(0, 5),
        "reg_lambda": uniform(0, 5)
    }
    xgb_search = RandomizedSearchCV(
        xg,
        xgb_param_dist,
        n_iter=XGB_ITER,
        scoring="neg_mean_squared_error",
        cv=tscv,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS_SEARCH,
        verbose=1
    )
    print("Running XGBoost RandomizedSearchCV (single-process)...")
    xgb_search.fit(X_train_num, y_train)
    best_xgb = xgb_search.best_estimator_
    xgb_pred = best_xgb.predict(X_test_num)
    xgb_rmse_tuned = mean_squared_error(y_test, xgb_pred)**0.5
    print(f"XGB tuned RMSE (H{h}): {xgb_rmse_tuned:.4f}")
    joblib.dump(best_xgb, os.path.join(TUNE_DIR, f"XGBoost_h{h}_tuned.joblib"))

    # record
    tuning_summary.append({
        "horizon": h,
        "prev_linear": prev_lin,
        "prev_rf": prev_rf,
        "prev_xgb": prev_xgb,
        "prev_naive": prev_naive,
        "rf_tuned_rmse": float(rf_rmse_tuned),
        "xgb_tuned_rmse": float(xgb_rmse_tuned),
        "rf_best_params": rf_search.best_params_,
        "xgb_best_params": xgb_search.best_params_
    })

# save tuning summary and build RMSE table
joblib.dump(tuning_summary, os.path.join(RESULT_DIR, "tuning_summary_fixed.joblib"))

# build final pivot table
rows = []
for t in tuning_summary:
    h = str(t["horizon"])
    rows.append(("Linear", h, t["prev_linear"]))
    rows.append(("Naive", h, t["prev_naive"]))
    rows.append(("RandomForest", h, t["rf_tuned_rmse"]))
    rows.append(("XGBoost", h, t["xgb_tuned_rmse"]))
summary_df = pd.DataFrame(rows, columns=["model", "horizon", "rmse"])
pivot = summary_df.pivot_table(index="model", columns="horizon", values="rmse", aggfunc='first')
pivot = pivot.reindex(index=["Linear", "Naive", "RandomForest", "XGBoost"])
pivot.to_csv(os.path.join(RESULT_DIR, "rmse_table_tuned_fixed.csv"))

print("\nTuning complete. Tuned models in:", TUNE_DIR)
print("Tuned RMSE table saved to:", os.path.join(RESULT_DIR, "rmse_table_tuned_fixed.csv"))



Tuning for horizon 1 hour(s)
Previous RMSEs (approx): Linear=17.4797, RF=18.1745, XGB=19.3884, Naive=30.6176
Running RandomForest RandomizedSearchCV (single-process)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
9 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/sklearn/base.py", line 1382, in wrapper
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/sklearn/base.py", line 436, in _validate_params
    call to `partial_fit`. All other methods that validate `X`
  File "/opt/

RF tuned RMSE (H1): 17.5546
Running XGBoost RandomizedSearchCV (single-process)...
Fitting 3 folds for each of 25 candidates, totalling 75 fits


KeyboardInterrupt: 

In [12]:
# tune_models_fast.py  (fast, laptop-friendly)
import os, joblib, numpy as np, pandas as pd
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.stats import randint, uniform

# CONFIG (fast)
DATA_PATH = "PRSA_Aotizhongxin_cleaned.csv"
TARGET = "PM2.5"
LAGS = 3
HORIZONS = [1, 3, 6, 12, 24]
TRAIN_FRAC = 0.8
RESULT_DIR = "results_multi_horizon"
TUNE_DIR = os.path.join(RESULT_DIR, "tuned_models_fast")
os.makedirs(TUNE_DIR, exist_ok=True)
RANDOM_STATE = 42

# FAST: reduce iterations & folds
RF_ITER = 8
XGB_ITER = 10
N_JOBS_SEARCH = 1      # safe single-process CV
N_JOBS_EST = 1
N_SPLITS = 2

def make_supervised(df, target_col, lags=3, horizon=1):
    df_sup = df.copy()
    for lag in range(1, lags + 1):
        df_sup = df_sup.assign(**{f"{c}_lag{lag}": df[c].shift(lag) for c in df.columns})
    df_sup[f"y_h{horizon}"] = df_sup[target_col].shift(-horizon)
    df_sup = df_sup.dropna()
    X = df_sup.drop(columns=[f"y_h{horizon}"])
    y = df_sup[f"y_h{horizon}"]
    return X, y

def train_test_split_time(X, y, train_frac=0.8):
    n = len(X)
    split = int(n * train_frac)
    return X.iloc[:split], X.iloc[split:], y.iloc[:split], y.iloc[split:]

def ensure_numeric(X_train, X_test):
    X_train = X_train.copy(); X_test = X_test.copy()
    obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    for c in obj_cols:
        X_train[c] = pd.to_numeric(X_train[c], errors='coerce')
        X_test[c]  = pd.to_numeric(X_test[c], errors='coerce')
        if X_train[c].dtype == 'object' or X_test[c].dtype == 'object':
            X_train[c] = X_train[c].map({'True':1,'False':0,True:1,False:0}).astype(float)
            X_test[c]  = X_test[c].map({'True':1,'False':0,True:1,False:0}).astype(float)
    X_train = X_train.fillna(0).astype(float)
    X_test  = X_test.fillna(0).astype(float)
    return X_train, X_test

# load df
df = pd.read_csv(DATA_PATH, parse_dates=True, index_col=0)

tuning_summary = []

for h in HORIZONS:
    print("\n" + "="*40)
    print(f"FAST tuning for horizon {h}h")
    X, y = make_supervised(df, TARGET, lags=LAGS, horizon=h)
    X_train, X_test, y_train, y_test = train_test_split_time(X, y, TRAIN_FRAC)
    X_train_num, X_test_num = ensure_numeric(X_train, X_test)

    # quick previous RMSEs using defaults (so we have a "before")
    naive_pred = X_test_num[f"{TARGET}_lag1"].values
    prev_naive = mean_squared_error(y_test, naive_pred)**0.5

    # ---------- RF fast search ----------
    rf = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS_EST)
    rf_param_dist = {
        "n_estimators": randint(100, 400),
        "max_depth": randint(5, 25),
        "min_samples_split": randint(2, 10),
        "min_samples_leaf": randint(1, 6),
        # REMOVE 'auto' — use allowed options only
        "max_features": ["sqrt", "log2", None, 0.3, 0.6]
    }
    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    rf_search = RandomizedSearchCV(
        rf, rf_param_dist, n_iter=RF_ITER,
        scoring="neg_mean_squared_error", cv=tscv,
        random_state=RANDOM_STATE, n_jobs=N_JOBS_SEARCH, verbose=1
    )
    rf_search.fit(X_train_num, y_train)
    best_rf = rf_search.best_estimator_
    rf_pred = best_rf.predict(X_test_num)
    rf_rmse = mean_squared_error(y_test, rf_pred)**0.5
    joblib.dump(best_rf, os.path.join(TUNE_DIR, f"RandomForest_h{h}_fast.joblib"))
    print(f" RF fast RMSE H{h}: {rf_rmse:.4f}")

    # ---------- XGB fast search ----------
    xg = xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS_EST, verbosity=0, objective='reg:squarederror')
    xgb_param_dist = {
        "n_estimators": randint(100, 400),
        "max_depth": randint(3, 8),
        "learning_rate": uniform(0.02, 0.3),
        "subsample": uniform(0.6, 0.4),
        "colsample_bytree": uniform(0.5, 0.4),
        "gamma": uniform(0, 2)
    }
    xgb_search = RandomizedSearchCV(
        xg, xgb_param_dist, n_iter=XGB_ITER,
        scoring="neg_mean_squared_error", cv=tscv,
        random_state=RANDOM_STATE, n_jobs=N_JOBS_SEARCH, verbose=1
    )
    xgb_search.fit(X_train_num, y_train)
    best_xgb = xgb_search.best_estimator_
    xgb_pred = best_xgb.predict(X_test_num)
    xgb_rmse = mean_squared_error(y_test, xgb_pred)**0.5
    joblib.dump(best_xgb, os.path.join(TUNE_DIR, f"XGBoost_h{h}_fast.joblib"))
    print(f" XGB fast RMSE H{h}: {xgb_rmse:.4f}")

    tuning_summary.append({
        "horizon": h,
        "prev_naive": float(prev_naive),
        "rf_fast_rmse": float(rf_rmse),
        "xgb_fast_rmse": float(xgb_rmse),
        "rf_best_params": rf_search.best_params_,
        "xgb_best_params": xgb_search.best_params_
    })

# save results
joblib.dump(tuning_summary, os.path.join(RESULT_DIR, "tuning_summary_fast.joblib"))

rows = []
for t in tuning_summary:
    h = str(t["horizon"])
    rows.append(("Naive", h, t["prev_naive"]))
    rows.append(("RandomForest", h, t["rf_fast_rmse"]))
    rows.append(("XGBoost", h, t["xgb_fast_rmse"]))
summary_df = pd.DataFrame(rows, columns=["model","horizon","rmse"])
pivot = summary_df.pivot_table(index="model", columns="horizon", values="rmse", aggfunc='first')
pivot = pivot.reindex(index=["Naive","RandomForest","XGBoost"])
pivot.to_csv(os.path.join(RESULT_DIR, "rmse_table_tuned_fast.csv"))
print("\nFAST tuning done. Results in:", TUNE_DIR)
print("Summary:", os.path.join(RESULT_DIR, "rmse_table_tuned_fast.csv"))



FAST tuning for horizon 1h
Fitting 2 folds for each of 8 candidates, totalling 16 fits
 RF fast RMSE H1: 17.6423
Fitting 2 folds for each of 10 candidates, totalling 20 fits
 XGB fast RMSE H1: 17.6682

FAST tuning for horizon 3h
Fitting 2 folds for each of 8 candidates, totalling 16 fits
 RF fast RMSE H3: 36.0534
Fitting 2 folds for each of 10 candidates, totalling 20 fits
 XGB fast RMSE H3: 37.1736

FAST tuning for horizon 6h
Fitting 2 folds for each of 8 candidates, totalling 16 fits
 RF fast RMSE H6: 50.6169
Fitting 2 folds for each of 10 candidates, totalling 20 fits
 XGB fast RMSE H6: 51.0720

FAST tuning for horizon 12h
Fitting 2 folds for each of 8 candidates, totalling 16 fits
 RF fast RMSE H12: 63.8022
Fitting 2 folds for each of 10 candidates, totalling 20 fits
 XGB fast RMSE H12: 63.8262

FAST tuning for horizon 24h
Fitting 2 folds for each of 8 candidates, totalling 16 fits
 RF fast RMSE H24: 71.9067
Fitting 2 folds for each of 10 candidates, totalling 20 fits
 XGB fast RM